In [18]:
import pandas as pd
import re
import numpy as np

In [19]:
def get_all_emails(text):
    EMAIL_REGEX = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""
    rows = text.split(sep='\n')
    emails = [re.search(EMAIL_REGEX, row.lower()) for row in rows]
    emails = [email.group() for email in emails if email]
    return emails


def get_one_email(row):
    EMAIL_REGEX = r'''(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])'''
    emails = re.search(EMAIL_REGEX, row.lower())
    if emails:
        return emails.group()


def get_telegram_contacts(row):
    TELEGRAM_NAME_REGEX = r'[\s:](@[A-Za-z0-9_]{3,20})' #\s or \b
    contacts = re.findall(TELEGRAM_NAME_REGEX, row.lower())
    return contacts


def get_mobile_numbers(row):
    MOBILE_NUMBER_REGEX = r'(?:\+7|\b7|\b8)(?:(?:[\D\(]){,2}\d){10}'
    mobile_numbers = re.findall(MOBILE_NUMBER_REGEX, row.lower())
    return mobile_numbers


def get_url_from_text(row):
    URL_REGEX = r'(https?:(?:\/{1,2}|\\{1,2})[^\s\(\)\[\]]+\w\b)'
    urls = re.findall(URL_REGEX, row.lower())
    return urls


def get_time_HHMM(row, need_replace=False):
    '''доработать "с 11:30 до 18 часов"'''
    TIME_REGEX = r'(?:[0-1]?\d|2[0-3]):[0-5]?\d.'
    if need_replace:
        row = re.sub(TIME_REGEX, 'TIME', row.lower())
        return row
    else:
        time_HHMM = re.findall(TIME_REGEX, row.lower())
        return time_HHMM


def get_salary(row):
    FROM_TO_REGEX = r'(?:от|до){0,1}\s{0,1}\d{1,4}[kкт\-\s$]{0,1}(?:\d+00+){1}'
    row_without_time_hhmm = get_time_HHMM(row.lower(), need_replace=True)
    salary = re.findall(FROM_TO_REGEX, row_without_time_hhmm)
    return salary

In [20]:
df = pd.read_csv('./data/main.csv')
df = df[df.salary.notna()]

In [21]:
df['test'] = df.text.apply(get_salary)
df[['salary', 'test']].sample(15)

,salary,test
3337,60-80 000,[80 000]
4710,200К - 250К,[]
6173,70К-110К,[]
4749,400-700к,[ 400-700]
5451,до 300к,[]
6274,до 150 000,[до 150 000]
2316,От 2000,[от 2000]
2476,150-275k,[]
5545,50 000-100 000,"[ 50 000, 100 000]"
3301,52 600,"[\n52 600, 2500, 3000]"


In [22]:
len(df[df.test.apply(len) == 0]) / len(df)

0.45024469820554647

In [23]:
test = df[df.test.apply(len) == 0]
test

,text,vacancy,IT,label,salary,test
2307,#Java #Spring Framework #spb\nНаш клиент- ком...,1,NaN,Java-разработчик,120-250к,[]
2337,"Менеджер проектов\nв INTECH — IT-компания, вед...",1,NaN,Project-менеджер,120К,[]
2366,Data Scientist (Pricing)\nМосква.\n300К.\n\nО ...,1,NaN,Data Scientist,300К,[]
2396,Технический директор\nв Ай-Теко — системный ин...,1,1.0,IT-директор,200К-250К,[]
2401,#вакансия #Android #fulltime #Тюмень\n\nAndroi...,1,NaN,Android-разработчик,от 90 до 170 т.,[]
...,...,...,...,...,...,...
6557,"Frontend (Vue, TypeScript) разработчик\nRemote...",1,NaN,Frontend-разработчик,170К — 210К,[]
6572,Data Scientist\nв ATI.SU — разрабатываем иннов...,1,NaN,Data Scientist,150К — 230К,[]
6574,#вакансия #СПБ \nИщу к себе в команду futureco...,1,NaN,HR-специалист,40K,[]
6583,#разработчик #JS #JavaScript #nodejs #moscow #...,1,NaN,Fullstack-разработчик,150-250k,[]


In [24]:
def get_salary_two(row):
    FROM_TO_REGEX = r'\b(?:от|до|\$){0,1}\s{0,1}\d{1,4} *(?:k|K|к|К|\$|,\d(?:k|k|к|К|$)| *т(?:ыс|\.))'
    row_without_time_HHMM = get_time_HHMM(row.lower(), need_replace=True)
    salary = re.findall(FROM_TO_REGEX, row_without_time_HHMM)
    return salary

In [25]:
 df.loc[df.test.apply(len) == 0, 'test'] = df[df.test.apply(len) == 0].text.apply(get_salary_two)

In [26]:
df.sample(30)

,text,vacancy,IT,label,salary,test
2379,Expert Java Developer/Архитектор Java\n\nКомпа...,1,1.0,Java-разработчик,250 000-350 000,"[ 250 000, 350 000]"
3851,"#jobs Вакансия: NLP, Machine learning специали...",1,NaN,Deep Learning Engineer,200-300К,[\n200-300]
3576,Frontend Team Lead\nв b2b-сервис для медиа и e...,1,1.0,Frontend-разработчик,250К,[от 250к]
3389,1.Ведущий программист 1с\n2. Иркутск\n3. 80 00...,1,NaN,1C-разработчик,80 000 -95 000,"[ 80 000, 95 000, 2000, 2000]"
3124,Senior/Lead DevOps\nМосква или удаленная работ...,1,1.0,"Инженер инфраструктуры, DevOps",До 400К,[до 400к]
3229,#вакансия #Москва #java #middle #senior #remot...,1,NaN,Java-разработчик,190k - 220k,"[190k, 220k, 290k, 310k]"
3660,Project Manager / Delivery Manager\nв Karuna —...,1,NaN,Project-менеджер,100К — 200К,"[100к, 200к]"
5065,• Python/Django разработчик / we⁢b developer (...,1,NaN,Python-разработчик,70 000 — 120 000,"[ 70 000, 120 000]"
2719,#вакансия #удаленно #удаленка #php #symfony #r...,1,NaN,PHP-разработчик,от 150.000 до 180.000,[]
3859,#вакансия #fulltime #remote #Москва #RubyOnRai...,1,NaN,Ruby-разработчик,от 100 000 до 250 000,"[от 100 000, до 250 000]"


In [27]:
len(test[test.test.apply(len) == 0]) / len(df)

0.45024469820554647